# Install Libraries

In [ ]:
!cat /proc/meminfo | grep MemTotal
!cat /proc/meminfo | grep MemAvailable

MemTotal:       13290460 kB
MemAvailable:   11948904 kB


In [ ]:
!pip install --upgrade transformers
!pip install --upgrade tf_keras
# then restrat session

# part 2

In [ ]:
!cat /proc/meminfo | grep MemTotal
!cat /proc/meminfo | grep MemAvailable

MemTotal:       13290460 kB
MemAvailable:   12147388 kB


In [ ]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [ ]:
import pandas as pd
df = pd.read_csv('dataset2_2.csv', delimiter='\t', header=None, names=['Sentence', 'Tags', 'Intent'])

In [ ]:
from transformers import BertTokenizer

model_name = "HooshvareLab/bert-fa-base-uncased"  # ParsBERT model
tokenizer = BertTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Encode sentences and tags
input_ids = []
attention_masks = []
tag_ids = []

# Creating tag encoders
tag2id = {tag: idx for idx, tag in enumerate(sorted(set(tag for tags in df['Tags'] for tag in tags.split())))}
id2tag = {idx: tag for tag, idx in tag2id.items()}
padding_tag_value = len(tag2id)
tag2id['PAD'] = padding_tag_value
id2tag[padding_tag_value] = 'PAD'

for i, row in df.iterrows():
    sentence = row['Sentence']
    tags = row['Tags'].split()

    # Tokenize sentence using BERT tokenizer
    bert_tokens = tokenizer.tokenize(sentence)
    input_id = tokenizer.convert_tokens_to_ids(bert_tokens)
    tag_id = [tag2id[tag] for tag in tags]

    input_ids.append(input_id)
    tag_ids.append(tag_id)
    attention_masks.append([1] * len(input_id))

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_LEN = 128
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", padding="post", truncating="post")
tag_ids = pad_sequences(tag_ids, maxlen=MAX_LEN, dtype="long", padding="post", truncating="post", value=padding_tag_value)
attention_masks = pad_sequences(attention_masks, maxlen=MAX_LEN, dtype="long", padding="post", truncating="post")

NameError: name 'input_ids' is not defined

In [ ]:
import numpy as np
# Convert to numpy arrays
input_ids = np.array(input_ids)
tag_ids = np.array(tag_ids)
attention_masks = np.array(attention_masks)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test, mask_train, mask_test = train_test_split(input_ids, tag_ids, attention_masks, test_size=0.2, random_state=42)

In [ ]:
from transformers import TFBertForTokenClassification

model = TFBertForTokenClassification.from_pretrained(model_name, num_labels=len(tag2id) + 1)

tf_model.h5:   0%|          | 0.00/963M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import tensorflow as tf

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': X_train, 'attention_mask': mask_train},
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': X_test, 'attention_mask': mask_test},
    y_test
))

# Batch the datasets
train_dataset = train_dataset.shuffle(len(X_train)).batch(16)
test_dataset = test_dataset.batch(16)


Datasets prepared successfully!


In [ ]:
from tensorflow.keras.optimizers import Adam

adam_optimizer = Adam(learning_rate=3e-5)

model.compile(optimizer=adam_optimizer,
              metrics=['accuracy'])


In [ ]:
history = model.fit(train_dataset, validation_data=test_dataset, epochs=3)

Epoch 1/3
675/675 [==============================] - 343s 443ms/step - loss: 0.0988 - accuracy: 0.9841 - val_loss: 0.0124 - val_accuracy: 0.9978
Epoch 2/3
675/675 [==============================] - 303s 449ms/step - loss: 0.0083 - accuracy: 0.9989 - val_loss: 0.0026 - val_accuracy: 0.9996
Epoch 3/3
675/675 [==============================] - 303s 449ms/step - loss: 0.0028 - accuracy: 0.9997 - val_loss: 0.0013 - val_accuracy: 0.9998


In [ ]:
# Save the model
model.save_pretrained("ParsBERT_BIO_Tagging_Model")
tokenizer.save_pretrained("ParsBERT_BIO_Tagging_Tokenizer")

('ParsBERT_BIO_Tagging_Tokenizer/tokenizer_config.json',
 'ParsBERT_BIO_Tagging_Tokenizer/special_tokens_map.json',
 'ParsBERT_BIO_Tagging_Tokenizer/vocab.txt',
 'ParsBERT_BIO_Tagging_Tokenizer/added_tokens.json')

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Predict on the test set
y_pred_prob = model.predict(test_dataset)
y_pred_indices = np.argmax(y_pred_prob.logits, axis=-1)

# Convert predictions and true labels to 1D arrays
y_pred_indices = y_pred_indices.reshape(-1)
y_test_flat = y_test.reshape(-1)

# Create mask to remove padding
mask = (y_test_flat != padding_tag_value)
y_pred_filtered = y_pred_indices[mask]
y_true_filtered = y_test_flat[mask]

# Map indices to tags
id2tag = {idx: tag for tag, idx in tag2id.items()}
y_pred_tags = [id2tag.get(index, 'OOV') for index in y_pred_filtered]
y_true_tags = [id2tag.get(index, 'OOV') for index in y_true_filtered]

# Find unique labels in predictions and true labels
unique_labels = set(y_true_tags + y_pred_tags)

# Define labels and target names
labels = sorted(unique_labels)

# Print classification report
print("Classification Report:")
print(classification_report(y_true_tags, y_pred_tags, labels=labels, target_names=labels))

# Print accuracy
accuracy = accuracy_score(y_true_tags, y_pred_tags)
print(f"Overall Accuracy: {accuracy:.4f}")


169/169 [==============================] - 33s 145ms/step
Classification Report:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

                        precision    recall  f1-score   support

              B-5G_yes       1.00      1.00      1.00       144
          B-5G_yes_not       0.44      1.00      0.62         4
                 B-RAM       1.00      1.00      1.00        58
                B-RAM1       0.98      1.00      0.99        56
            B-RAM1_not       1.00      0.50      0.67         2
                B-RAM2       0.95      1.00      0.97        53
            B-RAM2_not       0.00      0.00      0.00         4
                B-RAM3       1.00      1.00      1.00        45
            B-RAM3_not       1.00      1.00      1.00         4
             B-RAM_not       0.00      0.00      0.00         2
             B-RAMless       1.00      1.00      1.00        59
             B-RAMmore       0.97      1.00      0.99        68
    B-appearanceScore1       1.00      1.00      1.00       246
B-appearanceScore1_not       0.70      1.00      0.82         7
            B-batteryC       1.00      

In [ ]:
!cat /proc/meminfo | grep MemTotal
!cat /proc/meminfo | grep MemAvailable

MemTotal:       13290460 kB
MemAvailable:    8887920 kB


In [ ]:
tag2id

{'B-5G_yes': 0,
 'B-5G_yes_not': 1,
 'B-RAM': 2,
 'B-RAM1': 3,
 'B-RAM1_not': 4,
 'B-RAM2': 5,
 'B-RAM2_not': 6,
 'B-RAM3': 7,
 'B-RAM3_not': 8,
 'B-RAM_not': 9,
 'B-RAMless': 10,
 'B-RAMless_not': 11,
 'B-RAMmore': 12,
 'B-RAMmore_not': 13,
 'B-appearanceScore1': 14,
 'B-appearanceScore1_not': 15,
 'B-batteryC': 16,
 'B-batteryC1': 17,
 'B-batteryC1_not': 18,
 'B-batteryC2': 19,
 'B-batteryC2_not': 20,
 'B-batteryC3': 21,
 'B-batteryC3_not': 22,
 'B-batteryC_not': 23,
 'B-batteryCless': 24,
 'B-batteryCless_not': 25,
 'B-batteryCmore': 26,
 'B-batteryCmore_not': 27,
 'B-batteryLife1': 28,
 'B-batteryLife1_not': 29,
 'B-batteryLife2': 30,
 'B-batteryLife2_not': 31,
 'B-batteryLife3': 32,
 'B-batteryLife3_not': 33,
 'B-brand': 34,
 'B-brand1': 35,
 'B-brand1_not': 36,
 'B-brand2': 37,
 'B-brand2_not': 38,
 'B-brand_not': 39,
 'B-chipsetScore1': 40,
 'B-chipsetScore1_not': 41,
 'B-chipsetScore2': 42,
 'B-chipsetScore2_not': 43,
 'B-color': 44,
 'B-color1': 45,
 'B-color1_not': 46,
 'B-co

part 3

## **1 sentence - GRU**

In [ ]:
import time
import psutil
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import json
import pickle
import numpy as np

def monitor_memory():
    memory_usage = psutil.virtual_memory().used / (1024 ** 3)
    return memory_usage

def predict_gru_sentence(sentence):

    # Preprocess the input sentence
    sequence = tokenizer.texts_to_sequences([sentence])
    padded_sequence = pad_sequences(sequence, maxlen=max_len_sentences, padding='post')

    prediction = gru_model.predict(padded_sequence)

    predicted_tags = [np.argmax(p) for p in prediction[0]]
    predicted_tags = [list(tag_encoder.keys())[list(tag_encoder.values()).index(tag)] for tag in predicted_tags]


    return predicted_tags[:len(sentence.split())]


##############################
ram_before = monitor_memory()
##############################

# load model
gru_model = load_model('gru_bio_tagger_6.h5')

# load tokenizer
with open('bigru_tokenizer.json', 'r') as f:
    tokenizer_config = json.load(f)

tokenizer = Tokenizer() # Create a new Tokenizer object

tokenizer.word_index = tokenizer_config['word_index'] # Restore tokenizer's attributes
tokenizer.word_docs = tokenizer_config['word_docs']
tokenizer.index_docs = tokenizer_config['index_docs']

# Load the tag encoder from the file
with open('tag_encoder.pkl', 'rb') as f:
    tag_encoder = pickle.load(f)

max_len_sentences = 43

sentence = "من یک گوشی با صفحه نمایش کوچکتر از 4 می خواهم"

##############################
start_time = time.time()
##############################

predicted_tags = predict_gru_sentence(sentence)

##############################
end_time = time.time()
ram_after = monitor_memory()
##############################
total_time = end_time - start_time
ram_usage = ram_after - ram_before
##############################

print(f"Sentence: {sentence}")
print(f"Predicted Tags: {predicted_tags}")
print(f"Time Taken: {total_time:.4f} seconds")
print(f"RAM Usage: {ram_usage:.2f} GB")


## **20% - GRU**

In [ ]:
import time
import psutil
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import json
import pickle
import numpy as np

def monitor_memory():
    memory_usage = psutil.virtual_memory().used / (1024 ** 3)
    return memory_usage

def predict_gru_dataset(sentences):

    predictions = []
    for sentence in sentences:
      sequence = tokenizer.texts_to_sequences([sentence])
      padded_sequence = pad_sequences(sequence, maxlen=max_len_sentences, padding='post')
      prediction = gru_model.predict(padded_sequence)
      predicted_tags = [np.argmax(p) for p in prediction[0]]
      predicted_tags = [list(tag_encoder.keys())[list(tag_encoder.values()).index(tag)] for tag in predicted_tags]
      predictions.append((sentence, predicted_tags[:len(sentence.split())]))
    return predictions



##############################
ram_before = monitor_memory()
##############################

# load model
gru_model = load_model('gru_bio_tagger_6.h5')

# load tokenizer
with open('bigru_tokenizer.json', 'r') as f:
    tokenizer_config = json.load(f)

tokenizer = Tokenizer() # Create a new Tokenizer object

tokenizer.word_index = tokenizer_config['word_index'] # Restore tokenizer's attributes
tokenizer.word_docs = tokenizer_config['word_docs']
tokenizer.index_docs = tokenizer_config['index_docs']

# Load the tag encoder from the file
with open('tag_encoder.pkl', 'rb') as f:
    tag_encoder = pickle.load(f)

max_len_sentences = 43

# Load 20% of the dataset
df = pd.read_csv('dataset2_2.csv', delimiter='\t', header=None, names=['Sentence', 'Tags', 'Intent'])
sample_sentences = df['Sentence'].sample(frac=0.05).values
len_sample = len(sample_sentences)

##############################
start_time = time.time()
##############################

predictions = predict_gru_dataset(sample_sentences)

##############################
end_time = time.time()
ram_after = monitor_memory()
##############################
total_time = end_time - start_time
ram_usage = ram_after - ram_before
time_per_sentence = total_time / len_sample
##############################

print(f"Time Taken for 5% of the Dataset: {total_time:.4f} seconds")
print(f"Time per Sentence: {time_per_sentence:.4f} seconds")
print(f"RAM Usage: {ram_usage:.2f} GB")


## **1 sentence - parsbert**

In [ ]:
import time
import psutil
from transformers import BertTokenizer, TFBertForTokenClassification
import tensorflow as tf
import json
import pickle

def monitor_memory():
    memory_usage = psutil.virtual_memory().used / (1024 ** 3)
    return memory_usage

def predict_parsbert_sentence(sentence):

    # Preprocess the input sentence
    inputs = tokenizer(sentence, return_tensors="tf", padding=True, truncation=True, max_length=128)
    outputs = model(inputs)

    logits = outputs.logits
    predicted_ids = tf.argmax(logits, axis=-1).numpy().flatten()

    # Convert ids to tags
    id2tag = {idx: tag for tag, idx in tag2id.items()}
    predicted_tags = [id2tag.get(id, 'OOV') for id in predicted_ids]

    return predicted_tags


##############################
ram_before = monitor_memory()
##############################

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained("ParsBERT_BIO_Tagging_Tokenizer")
model = TFBertForTokenClassification.from_pretrained("ParsBERT_BIO_Tagging_Model")

# Load the tag encoder from the file
with open('tag_encoder.pkl', 'rb') as f:
    tag2id = pickle.load(f)

sentence = "من یک گوشی غیر آیفون و همچنین رم اش با رمی برابر با 3 گیگ می خواهم"

##############################
start_time = time.time()
##############################

predicted_tags = predict_parsbert_sentence(sentence)

##############################
end_time = time.time()
ram_after = monitor_memory()
##############################
total_time = end_time - start_time
ram_usage = ram_after - ram_before
##############################

print(f"Predicted Tags: {predicted_tags}")
print(f"Time Taken: {total_time:.4f} seconds")
print(f"RAM Usage: {ram_usage:.2f} GB")


## **20% - parsbert**

In [ ]:
import time
import psutil
from transformers import BertTokenizer, TFBertForTokenClassification
import tensorflow as tf
import json
import pickle
import pandas as pd

def monitor_memory():
    memory_usage = psutil.virtual_memory().used / (1024 ** 3)
    return memory_usage

def predict_parsbert_dataset(sentences):

    predictions = []
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="tf", padding=True, truncation=True, max_length=128)
        outputs = model(inputs)
        logits = outputs.logits
        predicted_ids = tf.argmax(logits, axis=-1).numpy().flatten()
        id2tag = {idx: tag for tag, idx in tag2id.items()}
        predicted_tags = [id2tag.get(id, 'OOV') for id in predicted_ids]
        predictions.append(predicted_tags)

    return predictions


##############################
ram_before = monitor_memory()
##############################

tokenizer = BertTokenizer.from_pretrained("ParsBERT_BIO_Tagging_Tokenizer")
model = TFBertForTokenClassification.from_pretrained("ParsBERT_BIO_Tagging_Model")

# Load 5% of the dataset
df = pd.read_csv('dataset2_2.csv', delimiter='\t', header=None, names=['Sentence', 'Tags', 'Intent'])
sample_sentences = df['Sentence'].sample(frac=0.05).values
len_sample = len(sample_sentences)

with open('tag_encoder.pkl', 'rb') as f:
    tag2id = pickle.load(f)

##############################
start_time = time.time()
##############################

predictions = predict_parsbert_dataset(sample_sentences)

##############################
end_time = time.time()
ram_after = monitor_memory()
##############################
total_time = end_time - start_time
ram_usage = ram_after - ram_before
time_per_sentence = total_time / len_sample
##############################

print(f"Time Taken for 5% of the Dataset: {total_time:.4f} seconds")
print(f"Time per Sentence: {time_per_sentence:.4f} seconds")
print(f"RAM Usage: {ram_usage:.2f} GB")

In [ ]:
import pandas as pd
df = pd.read_csv('dataset2_2.csv', delimiter='\t', header=None, names=['Sentence', 'Tags', 'Intent'])
df

,Sentence,Tags,Intent
0,من یک گوشی با صفحه نمایش کوچکتر از 4 می خواهم,O O O O O O O O B-displaySizeless O O,suggest_mobile
1,من یک گوشی با برندی به جز هوآوی می خواهم,O O O O O O O B-brand_not O O,suggest_mobile
2,من یک گوشی با حداکثر کیفیت دوربین 12 مگاپیکسل ...,O O O O O O O B-maincameraless O O O,suggest_mobile
3,من یک گوشی دوربینش حداقل برابر با 25 می خواهم,O O O O O O O B-maincameramore O O,suggest_mobile
4,من یک گوشی غیر آیفون و همچنین رم اش با رمی برا...,O O O O B-brand_not O O O O O O O O B-RAM O O O,suggest_mobile
...,...,...,...
13486,دوست ندارم با قابلیت شارژ سریع باشد,O O O O B-fastCharge_yes_not I-fastCharge_yes_...,suggest_mobile
13487,دوست ندارم دارای قلم باشد,O O O B-pen_yes_not O,suggest_mobile
13488,دوست ندارم با ظاهر شیک باشد,O O O O B-appearanceScore1_not O,suggest_mobile
13489,دوست ندارم دارای پردازنده خوب باشد,O O O O B-chipsetScore1_not O,suggest_mobile
